In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
from getpass import getpass
from urllib.parse import quote

import pandas as pd
from SPARQLWrapper import JSON, SPARQLWrapper
from tqdm.auto import tqdm

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import s3, iiif

try:
    S3_CLIENT_ID
except NameError:
    S3_CLIENT_ID = getpass("Please input s3 client id")

try:
    S3_CLIENT_SECRET
except NameError:
    S3_CLIENT_SECRET = getpass("Please input s3 client secret")


tqdm.pandas()
def identifier_to_base_url(identifier):
    return "https://media.performing-arts.ch/iiif/manifest/" + quote(identifier)


In [2]:
sparql_dev = SPARQLWrapper("https://www.dev.performing-arts.ch/sparql")

sparql_dev.setQuery(
    """
PREFIX spav: <http://vocab.performing-arts.ch/>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
SELECT ?instantiation WHERE {
  VALUES ?resource { <http://data.performing-arts.ch/r/7f45771c-ae90-4227-bc0c-c47b38ed10a2> }
  ?resource rico:hasInstantiation ?instantiation.
  ?instantiation rico:hasCarrierType spav:fvtif.
} ORDER BY ?instantiation
"""
)
sparql_dev.setReturnFormat(JSON)
results = sparql_dev.query().convert()

iiif_instantiation = pd.DataFrame.from_dict(
    [{k: v["value"] for k, v in x.items()} for x in results["results"]["bindings"]]
)

In [3]:
sparql = SPARQLWrapper("https://www.performing-arts.ch/sparql")

sparql.setQuery(
    """
PREFIX spav: <http://vocab.performing-arts.ch/>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
SELECT ?identifier ?unique_identifier_value ?name ?descriptive_note WHERE {
  VALUES ?resource { <http://data.performing-arts.ch/r/7f45771c-ae90-4227-bc0c-c47b38ed10a2> }
  OPTIONAL {
    ?resource rico:descriptiveNote ?descriptive_note.
  }
  OPTIONAL {
    ?resource rico:name ?name.
  }
   OPTIONAL {
    ?resource rico:identifier ?identifier.
  }
  OPTIONAL {
    ?resource rico:hasOrHadIdentifier ?unique_identifier.
    ?unique_identifier rdf:value ?unique_identifier_value;
                crm:P2_has_type spav:iduni.
  }
} ORDER BY ?identifier ?identifier_value
"""
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

metadata= pd.DataFrame.from_dict(
    [{k: v["value"] for k, v in x.items()} for x in results["results"]["bindings"]]
)

In [4]:
identifier = metadata['identifier'].iloc[0]
name = metadata['name'].iloc[0]
descriptive_note = metadata['descriptive_note'].iloc[0]
iiif_base_urls = [url.replace("http://", "https://") for url in iiif_instantiation['instantiation'].to_list()]

In [5]:
manifest_base_url = identifier_to_base_url(identifier)
manifest_base_metadata = iiif.get_manifest_base_metadata("Fondation Lifar", "Copyright : Fondation Lifar")
manifest = iiif.create_manifest_from_iiif_images(
    manifest_base_url,
    [iiif.IIIFImageItem(iiif_base_url, label=iiif_base_url.split("image%2F")[1], show_rendering=False) for iiif_base_url in iiif_base_urls],
    manifest_base_metadata,
    name,
    name,
    "http://data.performing-arts.ch/r/7f45771c-ae90-4227-bc0c-c47b38ed10a2",
    identifier,
    descriptive_note,
)

In [6]:
client = s3.S3Client(S3_CLIENT_ID, S3_CLIENT_SECRET)

client.upload_as_json(manifest, "manifests/" + identifier + ".json")

In [7]:
manifest['items']

[{'id': 'https://media.performing-arts.ch/iiif/manifest/1047-2-1-39-11/p001',
  'type': 'Canvas',
  'height': 3498,
  'width': 2842,
  'label': {'en': ['1047-2-1-39-11_001'],
   'de': ['1047-2-1-39-11_001'],
   'fr': ['1047-2-1-39-11_001'],
   'it': ['1047-2-1-39-11_001']},
  'thumbnail': [{'id': 'https://media.performing-arts.ch/iiif/3/image%2F1047-2-1-39-11_001/full/!300,300/0/default.jpg',
    'type': 'Image',
    'format': 'image/jpeg',
    'height': 300,
    'width': 300,
    'service': [{'id': 'https://media.performing-arts.ch/iiif/3/image%2F1047-2-1-39-11_001',
      'type': 'ImageService3',
      'profile': 'level2'}]}],
  'items': [{'id': 'https://media.performing-arts.ch/iiif/manifest/1047-2-1-39-11/p001/1',
    'type': 'AnnotationPage',
    'items': [{'id': 'https://media.performing-arts.ch/iiif/manifest/1047-2-1-39-11/annotation/p001-image',
      'type': 'Annotation',
      'motivation': 'painting',
      'body': {'id': 'https://media.performing-arts.ch/iiif/3/image%2F1047